### Threading
---

**Pros**

- Lightweight - low memory footprint
- Shared memory - makes access to state from another context easier
- Allows you to easily make responsive UIs
- cPython C extension modules that properly release the GIL will run in parallel
- Great option for I/O-bound applications

**Cons**

- cPython - subject to the GIL
- Not interruptible/killable
- If not following a command queue/message pump model (using the Queue module), then manual use of synchronization primitives become a necessity (decisions are needed for the granularity of locking)
- Code is usually harder to understand and to get right - the potential for race conditions increases dramatically

Source: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python

### Ejemplo1: Establecer comunicación entre procesos mediante la creación de un canal de comunicación
----



In [1]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://www.userena.digital/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

'http://www.foxnews.com/' page is 238856 bytes
'http://www.userena.digital/' page is 37380 bytes
'http://www.cnn.com/' page is 1894247 bytes
'http://www.bbc.co.uk/' page is 329359 bytes
'http://europe.wsj.com/' page is 992863 bytes


### Ejemplo 2: Consultas VO
----



In [2]:
import pyvo as vo
import concurrent.futures
import urllib.request
from concurrent.futures import ThreadPoolExecutor

In [3]:
URLS = ['http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/tap',
        'http://reg.g-vo.org/tap',
        'http://dc.g-vo.org/tap',
        'http://dc.zah.uni-heidelberg.de/tap',
       'https://vo.chivo.cl/tap']

In [4]:
url = "https://vo.chivo.cl/tap"
service = vo.dal.TAPService(url)

In [5]:
%%time
for url in URLS:
    service = vo.dal.TAPService(url)
    print("Url = "+url+" Disponible = "+str(service.available)+"Timeup ="+str(service.up_since))

Url = http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/tap Disponible = TrueTimeup =None
Url = http://reg.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://dc.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://dc.zah.uni-heidelberg.de/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = https://vo.chivo.cl/tap Disponible = TrueTimeup =2018-12-23T16:53:15Z
CPU times: user 37.6 ms, sys: 9.63 ms, total: 47.2 ms
Wall time: 3.05 s


In [6]:
def vo_tap(url):
    service = vo.dal.TAPService(url)
    print("Url = "+url+" Disponible = "+str(service.available)+"Timeup ="+str(service.up_since))
    return(service.available)

In [7]:
%%time
executor = ThreadPoolExecutor(max_workers=5)

task1 = executor.submit(vo_tap("https://vo.chivo.cl/tap"))
task2 = executor.submit(vo_tap("http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/tap"))
task3 = executor.submit(vo_tap("http://reg.g-vo.org/tap"))
task4 = executor.submit(vo_tap("http://dc.g-vo.org/tap"))
task5 = executor.submit(vo_tap("http://dc.zah.uni-heidelberg.de/tap"))

Url = https://vo.chivo.cl/tap Disponible = TrueTimeup =2018-12-23T16:53:15Z
Url = http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/tap Disponible = TrueTimeup =None
Url = http://reg.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://dc.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://dc.zah.uni-heidelberg.de/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
CPU times: user 37.4 ms, sys: 6.84 ms, total: 44.3 ms
Wall time: 2.79 s


In [8]:
%%time
future_to_url = {executor.submit(vo_tap, url): url for url in URLS}
for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]

Url = https://vo.chivo.cl/tap Disponible = TrueTimeup =2018-12-23T16:53:15Z
Url = http://dc.zah.uni-heidelberg.de/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://reg.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://dc.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/tap Disponible = TrueTimeup =None
CPU times: user 31.8 ms, sys: 7.12 ms, total: 38.9 ms
Wall time: 623 ms


In [9]:
for url in URLS:
    future_to_url = executor.submit(vo_tap, url)

Url = https://vo.chivo.cl/tap Disponible = TrueTimeup =2018-12-23T16:53:15Z
Url = http://reg.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://dc.zah.uni-heidelberg.de/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://dc.g-vo.org/tap Disponible = TrueTimeup =2019-03-14T15:18:19Z
Url = http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/tap Disponible = TrueTimeup =None


### Referencias
---

https://github.com/rsnemmen/parallel-python-tutorial/blob/master/Parallel%20Computing%20with%20Python%20public.ipynb

https://github.com/rsnemmen/parallel-python-tutorial

https://github.com/dask/dask-tutorial

https://docs.python.org/3/library/concurrent.futures.html